In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import os
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import zipfile
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.models as models
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers import MobileNetV1Config, MobileNetV1Model


In [ ]:
path = '../input/uavsdata/UAVs'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class_folders = os.listdir(path)
data = []  
labels = []
for class_folder in class_folders:
    class_path = os.path.join(path, class_folder)
    if os.path.isdir(class_path):  
        file_list = os.listdir(class_path)
        for img_name in tqdm(file_list):
            img_path = os.path.join(class_path, img_name)
            label = class_folder 
            try:
                img = cv2.imread(img_path)  
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
                img = cv2.resize(img, (224, 224))  
                data.append(img)  
                labels.append(label) 
            except Exception as e:
                pass

data = np.array(data)
labels = np.array(labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

In [ ]:
unique_labels = np.unique(labels)
fig, axs = plt.subplots(len(unique_labels), 1, figsize=(8, 8 * len(unique_labels)))

for i, label in enumerate(unique_labels):
    idx = np.where(labels == label)[0][0]
    axs[i].imshow(data[idx])
    axs[i].set_title(f"Class: {label}\nImage Size: {data[idx].shape}")
    axs[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
num_of_classes = LabelEncoder()
labels = num_of_classes.fit_transform(labels)
print(f"Total Number of Classes: {len(num_of_classes.classes_)}")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size=0.7)
print(f"X_train: {len(x_train)}  X_test: {len(x_test)}")

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [ ]:
batch_size=32
epoch = 20

train_dataset = CustomDataset(x_train, y_train, transform)
test_dataset = CustomDataset(x_test, y_test, transform)

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
r_cnn = resnet50(weights=ResNet50_Weights)
alexnet_pretrained = models.alexnet(pretrained=True)
google_cnn = models.googlenet(pretrained=True)
configuration = MobileNetV1Config()
pretrained_mobilenet = MobileNetV1Model(configuration)

r_cnn.to(device)
pretrained_mobilenet.to(device)
alexnet_pretrained.to(device)
google_cnn.to(device)

In [ ]:
def train_validate_model(model, train_loader, test_loader, criterion, optimizer, type_, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    train_predicted = []
    train_target = []
    test_predicted = []
    test_target = []
    train_precisions = []
    test_precisions = []
    train_recalls = []
    test_recalls = []
    train_f1scores = []
    test_f1scores = []
    
    early_stopping_iteration = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            if type_ == 2:
                loss = criterion(outputs.logits, labels) 
            else:
                loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            train_predicted.extend(predicted.cpu().numpy())
            train_target.extend(labels.cpu().numpy())

        train_loss = running_loss / len(trainloader)
        train_losses.append(train_loss)


        model.eval()
        correct = 0
        total = 0
        test_loss = 0.0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                if type_ == 2:
                    loss = criterion(outputs.logits, labels) 
                else:
                    loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                test_predicted.extend(predicted.cpu().numpy())
                test_target.extend(labels.cpu().numpy())

        test_loss /= len(testloader)
        test_losses.append(test_loss)

        train_accuracy = accuracy_score(train_target, train_predicted)
        test_accuracy = accuracy_score(test_target, test_predicted)
        train_precision = precision_score(train_target, train_predicted, average='weighted')
        test_precision = precision_score(test_target, test_predicted, average='weighted')
        train_recall = recall_score(train_target, train_predicted, average='weighted')
        test_recall = recall_score(test_target, test_predicted, average='weighted')
        train_f1score = f1_score(train_target, train_predicted, average='weighted')
        test_f1score = f1_score(test_target, test_predicted, average='weighted')

        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)
        train_precisions.append(train_precision)
        test_precisions.append(test_precision)
        train_recalls.append(train_recall)
        test_recalls.append(test_recall)
        train_f1scores.append(train_f1score)
        test_f1scores.append(test_f1score)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f},"
         f' Train Accuracy: {train_accuracy* 100:.2f}%, Test Accuracy: {test_accuracy* 100:.2f}%'
             f"Train Recall: {train_recall*100:.2f}%, Test Recall: {test_recall*100:2f}%")

    metrics = {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies,
        'train_predicted': train_predicted,
        'train_target': train_target,
        'test_predicted': test_predicted,
        'test_target': test_target,
        'train_precision': train_precisions,
        'test_precision': test_precisions,
        'train_recall': train_recalls,
        'test_recall':test_recalls,
        'train_f1score': train_f1score,
        'test_f1score': test_f1score
    }

    return metrics

In [ ]:
metrics = [{}, {}, {}, {}]
lr = [#0.001,
      #0.0001,
      0.0005,
      0.00005]
optimezers = ['Adam']
models = {
    0: "ResNet-50",
    1: "MobileNetV1",
    2: "AlexNet",
    3: "GoogleNet"
}

for optimizer_ in optimezers:
    for learning_rate in lr:
        if optimizer_ == 'Adam':            
            optimizer = optim.Adam(r_cnn.parameters(), lr=learning_rate)
            optimizer2 = optim.Adam(pretrained_mobilenet.parameters(), lr=learning_rate)
            optimizer3 = optim.Adam(alexnet_pretrained.parameters(), lr=learning_rate)
            optimizer4 = optim.Adam(google_cnn.parameters(), lr=learning_rate)
            criterion = nn.CrossEntropyLoss() 
            criterion2 = nn.CrossEntropyLoss()
            criterion3 = nn.CrossEntropyLoss() 
            criterion4 = nn.CrossEntropyLoss()

        elif optimizer_ == 'RMSProp':
            optimizer = optim.RMSProp(r_cnn.parameters(), lr=learning_rate)
            optimizer2 = optim.RMSProp(pretrained_mobilenet.parameters(), lr=learning_rate)
            optimizer3 = optim.RMSProp(alexnet_pretrained.parameters(), lr=learning_rate)
            optimizer4 = optim.RMSProp(google_cnn.parameters(), lr=learning_rate)
            criterion = nn.CrossEntropyLoss() 
            criterion2 = nn.CrossEntropyLoss()
            criterion3 = nn.CrossEntropyLoss() 
            criterion4 = nn.CrossEntropyLoss()
            
        elif optimizer_ == 'SGD':
            optimizer = optim.SGD(r_cnn.parameters(), lr=learning_rate)
            optimizer2 = optim.SGD(pretrained_mobilenet.parameters(), lr=learning_rate)
            optimizer3 = optim.SGD(alexnet_pretrained.parameters(), lr=learning_rate)
            optimizer4 = optim.SGD(google_cnn.parameters(), lr=learning_rate)
            criterion = nn.CrossEntropyLoss() 
            criterion2 = nn.CrossEntropyLoss()
            criterion3 = nn.CrossEntropyLoss() 
            criterion4 = nn.CrossEntropyLoss()
        
        metrics[0] = train_validate_model(r_cnn, trainloader, testloader, criterion, optimizer, 1, epoch)
        metrics[1] = train_validate_model(pretrained_mobilenet, trainloader, testloader, criterion,optimizer,2, epoch)
        metrics[2] = train_validate_model(alexnet_pretrained, trainloader, testloader, criterion, optimizer,3, epoch)
        metrics[3] = train_validate_model(google_cnn, trainloader, testloader, criterion, optimizer,4, epoch)
        
        for i in range (4):
            plt.figure(figsize=(20, 15))
            plt.subplot(3, 2, 1)
            plt.plot(metrics[i]['train_losses'], label='Train Loss')
            plt.plot(metrics[i]['test_losses'], label='Test Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'{models[i]} Losses, LR={learning_rate}, Optimizer={optimizer_}')
            plt.legend()

            plt.subplot(3, 2, 2)
            plt.plot(metrics[i]['test_accuracies'], label='Test Accuracy', color='b')
            plt.plot(metrics[i]['train_accuracies'], label='Train Accuracy', color='r')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'{models[i]} Accuracy, LR={learning_rate}, Optimizer={optimizer_}')
            plt.legend()
            
            plt.subplot(3, 2, 3)  
            plt.plot(metrics[i]['precisions'], label='Precision', color='g')
            plt.xlabel('Epoch')
            plt.ylabel('Precision, ')
            plt.title(f'{models[i]} Precision, LR={learning_rate}, Optimizer={optimizer_}')
            plt.legend()

            plt.subplot(3, 2, 4) 
            plt.plot(metrics[i]['recalls'], label='Recall', color='m')
            plt.xlabel('Epoch')
            plt.ylabel('Recall')
            plt.title(f'{models[i]} Recall, LR={learning_rate}, Optimizer={optimizer_}')
            plt.legend()

            plt.subplot(3, 2, 5) 
            plt.plot(metrics[i]['f1_scores'], label='F1 Score', color='y')
            plt.xlabel('Epoch')
            plt.ylabel('F1 Score')
            plt.title(f'{models[i]} F1 Score, LR={learning_rate}, Optimizer={optimizer_}')
            plt.legend()

        plt.tight_layout()
        plt.show()